In [1]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


In [2]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 38000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.03

len_sequence = 5e7 # DNA sequence length

n = 500 # number of generated   AF samples
n_neand = 10 #number of generated Neanderthals

rand_sd =1234 #random seed

T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF])/GEN_time

In [3]:
N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=20
ts3400 =sims.history_archaic(GEN_time, len_sequence, RR, MU, N_e, T,  n, rand_sd, n_neand,  
                          T_NEAND_samples/GEN_time, n_eu, N_EU_growth, 
                          T_EU_growth/GEN_time, N_EU_bottleneck, gr_rate, Portion_admix)
sims.print_neand_dosages(ts3400)

Neand ancestry:  [0.03136034, 0.0267591, 0.01943374, 0.03676582, 0.03129864, 0.0267383, 0.0132772, 0.02423768, 0.0294663, 0.02543408, 0.01992398, 0.0247902, 0.02288606, 0.02332984, 0.036199, 0.02056758, 0.02940606, 0.02469636, 0.02842558, 0.03006034]


In [4]:
ND_true_tracts = []
for idx in range(0, n_eu): 
    if (idx % 20) ==0:
        print('Done', idx)
    ND_true_tracts.append( sims.get_migrating_tracts_ind(ts3400, 'NEAND', idx, T[0]))      
   
s=0
for i in range(n_eu):
    for j in ND_true_tracts[i]:
        s+=j[1]-j[0]    
print('средняя доля неандертальца',s/(n_eu * len_sequence))

Done 0
средняя доля неандертальца 0.02625281


In [5]:
N_ref_pop=250
N_neanderthal=4
N=2
with open('make.obs.sims.txt', 'w') as f:
    f.write('#POSITIONS\t#REF\t#ALT\tANCESTRAL\t#OUTGROUP\t#ARCHAIC\t#OBSERVATIONS\n')
    for v in ts3400.variants():
        outgroup= str(list(set(v.genotypes[n_eu :( n_eu+N_ref_pop)]))).replace('[','').replace(']','').replace(' ','')
        archaic= str(list(set(v.genotypes[n_eu+n :( n_eu+n+N_neanderthal)]))).replace('[','').replace(']','').replace(' ','')

        obs=''
        for i in v.genotypes[0 :n_eu]:
            obs+=str(i)+' '



        f.write(str(int(v.site.position))+'\t'+str(v.alleles[0])+'\t'+
                str(v.alleles[1]) + '\t'+ str(0)+'\t' + outgroup+'\t'+archaic+'\t'+str(obs)+'\n')    

In [6]:
L=1000

#create bed file
with open('test.bed','w') as f:
    f.write('1\t0\t'+str(int(len_sequence)-1)+'\n')



f_bed='test.bed'
domain=useful.read_bed(f_bed)

n_windows=(domain[-1][1]-domain[0][0])//L + 1
windows_cover=np.ones(n_windows)*0.999

#create archaic covering file. 
CHR=1
with open('arch.covering.chr'+str(CHR)+'.txt','w') as f:
    for j in windows_cover:
        f.write(str(j)+'\n')


#create file with sample's names
with open('samples.txt','w') as f:
    for i in range(int(n_eu/2)):
        f.write('eu'+str(i)+'\n')

In [8]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM no --prepared_file make.obs.sims.txt --o out.txt --arch_cover arch.covering.chr1.txt

In [9]:
! python3 dai.seg.py --obs_samples samples.txt --bed test.bed   --HMM_par par.file.txt --EM yes --EM_steps 10 --prepared_file make.obs.sims.txt --o out.EM.txt --arch_cover arch.covering.chr1.txt

Число шагов в EM -алгоритме 8


In [10]:

ND_HMM_tracts=useful.read_out('out.txt')

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
df

,State,Value,Score,n_eu,n_neand,L,n_ref_pop
0,0,0.998134,precision,0,4,1000,250
1,0,0.999969,recall,0,4,1000,250
2,1,0.998995,precision,0,4,1000,250
3,1,0.942272,recall,0,4,1000,250
4,0,0.998923,precision,1,4,1000,250
...,...,...,...,...,...,...,...
75,1,0.890755,recall,18,4,1000,250
76,0,0.997826,precision,19,4,1000,250
77,0,0.999988,recall,19,4,1000,250
78,1,0.999566,precision,19,4,1000,250


In [11]:

ND_HMM_tracts=useful.read_out('out.EM.txt')

HMM_tracts=[]
REAL_tracts=[]
for idx in range(n_eu):
    HMM_tracts.append([useful.tracts_eu(ND_HMM_tracts[idx], len_sequence ), ND_HMM_tracts[idx]])
    REAL_tracts.append([ useful.tracts_eu(ND_true_tracts[idx], len_sequence), ND_true_tracts[idx]])

df2=useful.df_result(REAL_tracts, HMM_tracts, N_neanderthal, L, N_ref_pop, n_eu, N)
df2

,State,Value,Score,n_eu,n_neand,L,n_ref_pop
0,0,0.998178,precision,0,4,1000,250
1,0,0.999952,recall,0,4,1000,250
2,1,0.998420,precision,0,4,1000,250
3,1,0.943639,recall,0,4,1000,250
4,0,0.998923,precision,1,4,1000,250
...,...,...,...,...,...,...,...
75,1,0.903343,recall,18,4,1000,250
76,0,0.998032,precision,19,4,1000,250
77,0,0.999988,recall,19,4,1000,250
78,1,0.999569,precision,19,4,1000,250
